In [ ]:
properties = {}

properties['spark.app.name'] = "1B-kafka-postgres-stream"

properties['spark.sql.streaming.checkpointLocation'] = f"hdfs://172.21.1.190:8020/var/workspace/spark/app/checkpoint/{properties['spark.app.name']}"

properties['spark.stream.read.0A.kafka.bootstrap.servers'] = "172.21.1.110:39092"
properties['spark.stream.read.0A.subscribe']               = "kc_tradingexpert_fixlogtracer_ullink_ulbridge_bridge_v1"
properties['spark.stream.read.0A.startingOffsets']         = "earliest"
properties['spark.stream.read.0A.maxOffsetsPerTrigger']    = 10000
properties['spark.stream.read.0A.kafka.group.id']          = f"{properties['spark.app.name']}"

properties['spark.stream.write.1A.kafka.bootstrap.servers'] = "172.21.1.110:39092"
properties['spark.stream.write.1A.topic']                   = f"{properties['spark.app.name']}"

properties['spark.stream.write.1B.url']                     = "jdbc:postgresql://172.21.1.110:5432/kc_tradingexpert_fixorderlinker"
properties['spark.stream.write.1B.dbtable']                 = f"{properties['spark.app.name']}"
properties['spark.stream.write.1B.user']                    = "admin"
properties['spark.stream.write.1B.password']                = "adminadmin123456"


properties['spark.driver.memory']                        = "512m"

properties

In [ ]:
from kc.annotation.connectivity import ConnectivitySpark

In [ ]:
@ConnectivitySpark(configuration={
    "spark.app.name": properties['spark.app.name'],
    "spark.sql.streaming.checkpointLocation": properties['spark.sql.streaming.checkpointLocation'],
    "spark.driver.memory": properties['spark.driver.memory']
})
class Application:
    pass

In [ ]:
app = Application()

In [ ]:
###################################################################################################################

In [ ]:
import psycopg2
from psycopg2 import Error

In [ ]:
postgres_create_table_body = """
CREATE TABLE IF NOT EXISTS app_biet_513_4_smu_2T (
    "__spec__source_CLORDID"                      VARCHAR(100),
    "__spec__sink_CLORDID"                        VARCHAR(100),
    "__spec__CLORDIDs"                            TEXT [],
    "__source__KAFKA_SOURCE_MESSAGE_TIMESTAMP"    TIMESTAMP,
    "__sink__KAFKA_SOURCE_MESSAGE_TIMESTAMP"      TIMESTAMP,
    "__KAFKA_SOURCE_MESSAGE_TIMESTAMPs"           TIMESTAMP [], 
    "__message_type"                              VARCHAR(100)
);
"""

In [ ]:
try:
    connection = psycopg2.connect(
                    host="172.21.1.110",
                    port="5432",
                    database="kc_tradingexpert_fixorderlinker",
                    user="admin",
                    password="adminadmin123456")
    cursor = connection.cursor()
    cursor.execute(postgres_create_table_body)
    cursor.close()
    connection.commit()
except(Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        connection.close()
        print("PostgresSQL connection is closed")

In [ ]:
###################################################################################################################

In [ ]:
postgres_query_body = """
SELECT
    *
FROM
    app_biet_513_4_smu_2T
"""

In [ ]:
df_source_postgres = app.spark.read.format("jdbc") \
                                   .option("driver", "org.postgresql.Driver") \
                                   .option("url", "jdbc:postgresql://172.21.1.110:5432/kc_tradingexpert_fixorderlinker") \
                                   .option("user", "admin") \
                                   .option("password", "adminadmin123456") \
                                   .option("query", postgres_query_body) \
                                   .load()
df_source_postgres.createOrReplaceTempView("df_source_postgres")

In [ ]:
###################################################################################################################

In [ ]:
import pyspark.sql

def __DumpBatchDF_SinkGroup_2(batch_DF: pyspark.sql.DataFrame, batch_id: int) -> None:
    batch_DF.persist()
    batch_DF.write.format("jdbc") \
            .option("driver", "org.postgresql.Driver") \
            .option("url", properties['spark.stream.write.1B.url']) \
            .option("dbtable", properties['spark.stream.write.1B.dbtable']) \
            .option("user", properties['spark.stream.write.1B.user']) \
            .option("password", properties['spark.stream.write.1B.password']) \
            .mode("append") \
            .save()
    batch_DF.unpersist()

In [ ]:
###################################################################################################################

In [ ]:
pipeline = df_source_postgres.writeStream \
                  .foreachBatch(__DumpBatchDF_SinkGroup_2) \
                  .outputMode("append") \
                  .trigger(processingTime='1 second') \
                  .start()

In [ ]:
###################################################################################################################